In [6]:
import chess
import chess.svg
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import time  # To see AI thinking time
from nnue.minimax import MinimaxNNUE
from knightvision.minimax_inter import KnightVision
from pathlib import Path
model_path =  "./nnue/nnue_3_1978880d_512bs_200es_51e.pth"
# chessAI = MinimaxNNUE(depth=3, path_file=str(model_path))
chessAI = KnightVision(time_limit=2.0)
test_cases = [
    # Checkmate/Tactics Cases
    {"name": "Checkmate in 1 (Scholar's)", "fen": "rnbqkbnr/p1pppppp/8/8/1pB1P3/5Q2/PPPP1PPP/RNB1K1NR w KQkq - 0 4"},
    {"name": "Trap (Fried Liver setup)", "fen": "r1bqkb1r/pppp1ppp/2n2n2/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR w KQkq - 4 4"}
]

def display_svg_board(board, size=300): # Reduced size for side-by-side
    """Displays a chessboard using SVG."""
    return chess.svg.board(board, size=size)

# Dropdown to select the test case
test_case_names = [tc["name"] for tc in test_cases]
dropdown = widgets.Dropdown(
    options=test_case_names,
    description='Test Case:',
    disabled=False,
)

# Output area to display the boards and information
output_area = widgets.Output()

def on_dropdown_change(change):
    """This function runs when the dropdown value changes."""
    with output_area: # Direct output to the output_area widget
        clear_output(wait=True) # Clear previous output smoothly

        selected_name = change['new']
        selected_case = next((tc for tc in test_cases if tc["name"] == selected_name), None)

        if not selected_case:
            print(f"Error: Test case '{selected_name}' not found.")
            return

        fen = selected_case["fen"]
        initial_board = chess.Board(fen)

        display(HTML(f"<h3>Test Case: {selected_name}</h3>"))
        display(HTML(f"<b>Initial FEN:</b> {fen}"))

        # --- Run the AI ---
        display(HTML("<p><i>Running AI...</i></p>"))
        start_time = time.time()
        try:
            best_move = chessAI.find_best_move(initial_board)
            end_time = time.time()
            duration = end_time - start_time
            display(HTML(f"<p><i>AI finished in {duration:.2f} seconds.</i></p>"))

            if initial_board.is_game_over():
                display(HTML(f"<b>Game is already over: {initial_board.result()}</b>"))
                best_move = None # No move to make if game is over

            elif best_move:
                # Check if the move is legal before applying
                if best_move in initial_board.legal_moves:
                    ai_board = initial_board.copy() # Create a copy to apply the move
                    ai_board.push(best_move) # Apply the move

                    # Display initial and final boards side-by-side using HTML table
                    board_display = f"""
                    <table>
                        <tr>
                            <td style="text-align: center;"><b>Initial Board:</b><br>{display_svg_board(initial_board)}</td>
                            <td style="text-align: center;"><b>AI Move:</b> {initial_board.san(best_move)} ({best_move.uci()})<br><b>Board After AI Move:</b><br>{display_svg_board(ai_board)}</td>
                        </tr>
                    </table>
                    """
                    display(HTML(board_display))
                    display(HTML(f"<b>FEN After Move:</b> {ai_board.fen()}"))

                    if ai_board.is_checkmate():
                        display(HTML("<b style='color:red;'>CHECKMATE!</b>"))
                    elif ai_board.is_stalemate():
                        display(HTML("<b style='color:orange;'>STALEMATE!</b>"))
                    elif ai_board.is_insufficient_material():
                        display(HTML("<b style='color:grey;'>INSUFFICIENT MATERIAL!</b>"))
                    elif ai_board.is_check():
                        display(HTML("<b style='color:blue;'>CHECK!</b>"))

                else:
                    display(HTML(f"<b style='color:red;'>Error: AI proposed an illegal move: {best_move.uci()}</b>"))
                    # Optionally display legal moves for debugging:
                    # display(HTML(f"Legal moves were: {', '.join(m.uci() for m in board.legal_moves)}"))

            else:
                display(HTML("<b style='color:orange;'>AI found no move (or decided not to move?).</b>"))
                if initial_board.is_stalemate():
                    display(HTML("<b style='color:orange;'>Reason: STALEMATE!</b>"))
                elif initial_board.is_checkmate(): # Should technically not happen if no move found, but good check
                    display(HTML("<b style='color:red;'>Reason: CHECKMATE!</b>"))


        except Exception as e:
            display(HTML(f"<b style='color:red;'>Error during AI execution: {e}</b>"))

        display(HTML("<hr>")) # Separator

dropdown.observe(on_dropdown_change, names='value')

display(dropdown)
display(output_area)

# Trigger the display for the initial dropdown value
on_dropdown_change({'new': dropdown.value})

Dropdown(description='Test Case:', options=("Checkmate in 1 (Scholar's)", 'Trap (Fried Liver setup)'), value="…

Output()